# Explicacion de "Generative Pre-trained Transformer" (GPT) Entrenado

colocari introduccion

🧠 Explicación paso a paso del algoritmo GPT (versión resumida)

---

1. Ejecución principal (`run.py`)

- Es el punto de entrada del programa.
- Usa un argumento `mode` para decidir qué acción ejecutar:
  - **`preprocess`** → prepara los datos de texto.
  - **`train`** → entrena o actualiza el modelo GPT.
  - **`chat`** → inicia una conversación interactiva con el modelo.

---

2. Preprocesamiento de datos (`preprocess.py`)

1. Lee el archivo de chat (`assets/input/chat.txt`).
2. Elimina caracteres poco frecuentes (como emojis o símbolos raros).
3. Divide los mensajes en tuplas: *(fecha, contacto, mensaje)*.
4. Crea *tokens especiales* con los nombres de los contactos y el token de fin `<END>`.
5. Tokeniza el texto respetando los tokens especiales.
6. Reemplaza tokens muy raros por `<UNK>` para reducir el vocabulario.
7. Genera el **vocabulario** (lista única de tokens).
8. Codifica los tokens a tensores numéricos.
9. Divide los datos en:
   - **Entrenamiento:** 90%
   - **Validación:** 10%
10. Guarda los tensores y archivos auxiliares:
    - `train.pt`, `valid.pt`, `vocab.txt`, `contacts.txt`.

---

3. Definición del modelo (`model.py`)

Implementa una versión simplificada de **GPT (Transformer Decoder)** con los siguientes componentes:

- **`Head`** → Aplica *self-attention* sobre el contexto.
- **`MultiHeadAttention`** → Ejecuta varias cabezas de atención en paralelo.
- **`FeedForward`** → Red densa que transforma la salida de la atención.
- **`Block`** → Combina atención y red feed-forward con normalización residual.
- **`GPTLanguageModel`** → Une todos los bloques, agrega embeddings y produce predicciones finales.

Incluye un método:
- **`generate()`** → Genera texto token por token hasta encontrar el token `<END>`.

---

4. Entrenamiento del modelo (`train.py`)

1. Carga los tensores de entrenamiento y validación.
2. Carga el vocabulario (`vocab.txt`).
3. Si `--update` está activado:
   - Carga un modelo existente (`model.pt`) para continuar su entrenamiento.
   - Si no, crea un modelo nuevo desde cero.
4. Define el optimizador **AdamW** con una tasa de aprendizaje (`learn_rate`).
5. Ciclo de entrenamiento (`max_iters` iteraciones):
   - Obtiene un **batch** de datos con `get_batch()`.
   - Calcula la **pérdida (loss)** entre predicciones y objetivos.
   - Realiza retropropagación y actualiza los pesos.
   - Cada `eval_interval` pasos, evalúa la pérdida en *train* y *validación* (`estimate_loss`).
6. Guarda el modelo entrenado en `assets/models/model.pt`.

---

5. Chat interactivo (`chat.py`)

1. Carga el modelo GPT entrenado y el vocabulario.
2. Pide al usuario un mensaje de entrada.
3. El modelo genera una respuesta token a token.
4. Muestra la salida usando `print_delayed()` (efecto de escritura lenta).
5. Continúa la conversación hasta que el usuario escribe `<END>`.

---

6. Funciones auxiliares (`utils.py`)

| Función | Descripción |
|----------|--------------|
| `encode` / `decode` | Convierte entre texto y tensores de índices. |
| `get_batch` | Crea lotes de secuencias para el entrenamiento. |
| `estimate_loss` | Calcula la pérdida promedio del modelo. |
| `custom_tokenizer` | Tokeniza texto conservando tokens especiales. |
| `get_vocab` | Genera una lista de tokens únicos. |
| `print_delayed` | Imprime texto con un efecto de tipeo. |
| `current_time` | Muestra la hora actual (para logs). |

---

⚙️ Resumen general

1. **Entrada:** Chat de WhatsApp en texto plano.  
2. **Preprocesamiento:** Limpieza, tokenización y codificación numérica.  
3. **Entrenamiento:** Ajuste de un modelo GPT (transformer decoder) con esos datos.  
4. **Generación:** El modelo aprende a responder mensajes en el mismo estilo.  
5. **Interfaz:** Chat interactivo donde el modelo responde automáticamente.

---

> 💬 En síntesis, este proyecto implementa un mini-GPT capaz de aprender patrones de conversación a partir de chats reales, y luego simular respuestas coherentes y personalizadas.


## run.py

El archivo run.py es el punto de entrada principal del proyecto y funciona como una interfaz de línea de comandos (CLI) que permite ejecutar las distintas fases del pipeline del modelo GPT: el preprocesamiento de datos, el entrenamiento y el modo de conversación. Para lograr esto, utiliza el módulo estándar de Python llamado argparse, que se encarga de interpretar los argumentos que el usuario escribe al ejecutar el script. En las primeras líneas se importan los módulos necesarios: argparse para gestionar los argumentos y los tres submódulos del proyecto ubicados en src —chat, preprocess y train—, cada uno responsable de una parte distinta del proceso.
La función principal main() comienza creando un objeto ArgumentParser que define los parámetros que el usuario puede pasar por consola. Luego, agrega un argumento posicional obligatorio llamado "mode", que solo acepta tres valores posibles: "preprocess", "train" o "chat". Este argumento determina qué acción realizará el programa. Además, se define un argumento opcional --update, que actúa como una bandera booleana (True si se incluye en el comando y False en caso contrario). Este flag se usa para indicar si, durante el entrenamiento, el modelo debe continuar a partir de un modelo previamente guardado en lugar de iniciar desde cero.
Una vez definidos los argumentos, se ejecuta parser.parse_args(), que interpreta los parámetros escritos en la línea de comandos y los guarda en un objeto args. A continuación, mediante una estructura condicional, el programa ejecuta la función correspondiente según el valor de args.mode: si el usuario eligió "preprocess", se llama a preprocess.make_train_test(), que se encarga de tokenizar el texto, generar el vocabulario y crear los conjuntos de entrenamiento y validación; si eligió "train", se invoca a train.model_training(args.update), que entrena el modelo (desde cero o continuando, dependiendo de la bandera --update); y si eligió "chat", se llama a chat.conversation(), que carga el modelo entrenado y permite mantener un diálogo con él generando texto de forma autoregresiva.
Finalmente, el bloque if __name__ == "__main__": main() asegura que esta función se ejecute solo cuando el archivo se ejecuta directamente desde la terminal (por ejemplo, python run.py train) y no cuando es importado desde otro módulo. En conjunto, run.py actúa como un controlador o “menú principal” que organiza el flujo completo del proyecto. Gracias a esta estructura, el usuario puede realizar todas las operaciones principales —preprocesamiento, entrenamiento y conversación— mediante simples comandos en la terminal, sin necesidad de modificar el código fuente.

In [ ]:
import argparse

from src import chat, preprocess, train


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("mode", choices=["preprocess", "train", "chat"], help="The mode to be execute.")
    parser.add_argument("--update", action="store_true", help="Flag when model shall be updated based on current parameters")
    args = parser.parse_args()

    if args.mode == "preprocess":
        preprocess.make_train_test()
    elif args.mode == "train":
        train.model_training(args.update)
    elif args.mode == "chat":
        chat.conversation()

if __name__ == "__main__":
    main()


## config.py

El archivo config.py actúa como un contenedor central de parámetros y configuraciones que controlan el comportamiento del modelo, del entrenamiento y del preprocesamiento. Su función es evitar que los valores clave estén “duros” dentro del código, lo que permite modificar la arquitectura o los hiperparámetros de forma simple y coherente desde un solo lugar. En este caso, el archivo define tres grupos principales de configuraciones: parámetros del modelo, parámetros del entrenamiento y parámetros del preprocesamiento o codificación.

En la primera sección, “model hyperparameters”, se establecen los valores que determinan la estructura del modelo tipo GPT.

block_size = 32 indica la longitud máxima del contexto o ventana de tokens que el modelo usa para predecir el siguiente elemento. En otras palabras, el modelo solo “ve” los últimos 32 tokens del texto a la vez, lo que define su capacidad de memoria a corto plazo.

embed_size = 256 define la dimensión del espacio vectorial en el que se representarán los tokens después del proceso de embedding; cada palabra o símbolo del vocabulario se convierte en un vector de 256 valores.

dropout = 0.2 es la tasa de abandono utilizada para evitar el sobreajuste, lo que significa que el 20 % de las conexiones del modelo se desactivan aleatoriamente durante el entrenamiento.

n_heads = 6 especifica que la atención multi-cabeza del Transformer estará compuesta por seis cabezas, lo que permite que el modelo atienda simultáneamente a distintas partes del contexto.

n_layer = 6 indica que el modelo tendrá seis bloques o capas de Transformer apilados, cada uno compuesto por mecanismos de atención y redes feed-forward.

eval_iters = 200 indica cuántos lotes (batches) se utilizarán durante la evaluación para estimar la pérdida promedio en los conjuntos de entrenamiento y validación.

batch_size = 32 determina cuántas secuencias se procesan en paralelo en cada iteración de entrenamiento.

En la segunda sección, “learning hyperparameters”, se definen los parámetros que afectan directamente la dinámica del entrenamiento.

learn_rate = 3e-4 (0.0003) es la tasa de aprendizaje del optimizador, que controla el tamaño de los pasos en la actualización de los pesos.

max_iters = 5000 representa el número máximo de iteraciones de entrenamiento (pasos de optimización).

eval_interval = 500 indica que cada 500 iteraciones se realizará una evaluación del modelo en los conjuntos de datos de entrenamiento y validación para monitorear su desempeño.

La tercera parte, “preprocess”, contiene configuraciones del procesamiento del texto.

min_count_chars = 1 y min_count_tokens = 1 establecen la frecuencia mínima que debe tener un carácter o token en el corpus para ser incluido en el vocabulario. En este caso, cualquier token que aparezca al menos una vez se mantiene, lo que equivale a no filtrar por frecuencia.

Finalmente, en la sección “encoding”, se definen los tokens especiales usados para representar situaciones particulares dentro del texto.

end_token = "<END>" marca el final de una secuencia o conversación, ayudando al modelo a entender dónde termina una respuesta.

unknown_token = "<UNK>" se usa para reemplazar cualquier palabra o símbolo que no esté en el vocabulario (token desconocido).

n_chats = 5 probablemente determina el número máximo de turnos de conversación o la cantidad de mensajes previos que el modelo puede considerar en modo “chat”.

En conjunto, este archivo config.py proporciona la base que guía la construcción del modelo, su entrenamiento y el tratamiento de los datos. Centralizar estos valores facilita la reproducibilidad de los experimentos y permite ajustar rápidamente la complejidad del modelo o la escala del entrenamiento sin modificar el código fuente de los módulos principales.

In [ ]:
# model hyperparameters
block_size = 32
embed_size = 256
dropout = 0.2
n_heads = 6
n_layer = 6
eval_iters = 200
batch_size = 32

# learning hyperparameters
learn_rate = 3e-4
max_iters = 5000
eval_interval = 500

# preprocess
min_count_chars = 1
min_count_tokens = 1

# encoding
end_token = "<END>"
unknown_token = "<UNK>"
n_chats = 5


## chat.py

El archivo chat.py implementa el modo de conversación del modelo GPT previamente entrenado, simulando un chat interactivo entre el usuario y el modelo. Su objetivo es cargar el modelo entrenado y su vocabulario, luego generar respuestas automáticamente a los mensajes que el usuario escriba, manteniendo un contexto de diálogo.

El script comienza importando varios módulos. json se utiliza para leer los archivos que contienen el vocabulario y la lista de contactos; random permite seleccionar aleatoriamente un contacto durante la simulación; y torch es la librería principal para cargar el modelo de lenguaje entrenado y manejar tensores. Luego, se importa prompt y WordCompleter desde prompt_toolkit, una librería que mejora la interacción en consola, ofreciendo autocompletado de palabras mientras el usuario escribe (por ejemplo, los nombres de contactos o el token de fin). Posteriormente se importan desde config.py los valores end_token (el token que indica el final de la conversación) y n_chats (el número de turnos automáticos que el modelo genera antes de volver a pedir entrada al usuario). Finalmente, desde src.utils se traen las funciones auxiliares:

custom_tokenizer: tokeniza el texto de entrada teniendo en cuenta los tokens especiales,

encode y decode: convierten entre texto y tensores numéricos según el vocabulario,

print_delayed: imprime texto de manera pausada, simulando que el modelo “escribe” como una persona.

La función principal, conversation(), es la encargada de orquestar el chat. Primero abre y lee el archivo "assets/output/vocab.txt", que contiene el vocabulario del modelo en formato JSON, y lo carga como un diccionario vocab. Luego hace lo mismo con "assets/output/contacts.txt", una lista de posibles nombres o interlocutores del chat, guardándolos en contacts. A partir de esta información, crea la lista spec_tokens, que combina los nombres de contacto y el token especial de fin (<END>). A continuación, se carga el modelo de lenguaje con torch.load("assets/models/model.pt"), lo que restaura el estado del modelo entrenado (una instancia de GPTLanguageModel). También se crea un objeto WordCompleter con spec_tokens, que permite que el autocompletado en la consola sugiera automáticamente los contactos o el token <END> cuando el usuario escribe.

El primer prompt solicita al usuario un mensaje inicial con la etiqueta "message >> ". El texto ingresado se almacena en la variable input. A la vez, se crea un tensor vacío llamado output, que representará la secuencia de tokens generada por el modelo durante la conversación.

A partir de ahí, se entra en un bucle while input != end_token: que se ejecuta hasta que el usuario escriba el token <END>, lo que detiene la conversación. Dentro del bucle, hay otro ciclo for _ in range(n_chats): que controla cuántas respuestas automáticas generará el modelo antes de volver a solicitar una entrada del usuario (por ejemplo, si n_chats = 5, el modelo generará cinco mensajes seguidos).

En cada iteración del ciclo, el texto del usuario (input) se pasa a custom_tokenizer, que lo divide en tokens respetando los tokens especiales como los nombres de contacto o <END>. Luego, con encode(add_tokens, vocab), esos tokens se convierten a índices numéricos según el vocabulario cargado, obteniendo un tensor add_context. Este tensor se concatena con output (que acumula el historial de conversación) mediante torch.cat((output, add_context)), y se reorganiza con .unsqueeze(1).T para ajustar sus dimensiones al formato que espera el modelo: una secuencia de tamaño (1, longitud) en lugar de (longitud,).

Después, se guarda la longitud actual de output en n0, y se llama a model.generate(context, vocab), que usa el modelo para generar nuevos tokens a partir del contexto actual. El resultado, un tensor más largo que contiene la nueva predicción, se guarda nuevamente en output. La diferencia n1 - n0 permite identificar cuántos tokens nuevos fueron generados. Con decode(output[n0-n1:], vocab) se transforman esos tokens nuevos en texto legible, y print_delayed los imprime en pantalla simulando la escritura progresiva del modelo. Para el siguiente turno, la variable input se actualiza eligiendo aleatoriamente un nuevo contacto desde la lista contacts, lo que crea la sensación de que el modelo conversa con distintos interlocutores.

Una vez que el bucle de n_chats termina, el programa vuelve a pedir una nueva entrada del usuario mediante prompt("\nresponse >> ", completer=completer, default=""). Este nuevo mensaje se procesa nuevamente en el siguiente ciclo while. Si el usuario introduce el token <END>, la condición input != end_token deja de cumplirse y la conversación se detiene.

En resumen, chat.py convierte el modelo GPT entrenado en una experiencia de chat interactiva en consola. Lee el vocabulario y los contactos, carga el modelo, permite que el usuario escriba mensajes con autocompletado y genera respuestas automáticas en base al contexto previo. Su estructura combina procesamiento de texto, generación autoregresiva con el modelo y una interfaz de usuario sencilla, todo ello gestionado en un ciclo continuo que se detiene solo cuando el usuario decide finalizar la conversación.

In [ ]:
import json
import random

import torch
from prompt_toolkit import prompt
from prompt_toolkit.completion import WordCompleter

from config import end_token, n_chats
from src.utils import custom_tokenizer, decode, encode, print_delayed


def conversation() -> None:
    """
    Emulates chat conversations by sampling from a pre-trained GPTLanguageModel.

    This function loads a trained GPTLanguageModel along with vocabulary and 
    the list of special tokens. It then enters into a loop where the user specifies 
    a contact. Given this input, the model generates a sample response. The conversation 
    continues until the user inputs the end token.

    :example:

    >>> conversation()
    message >> Alice
    Model's Response: How are you?
    response >> end
    """
    with open("assets/output/vocab.txt", "r", encoding="utf-8") as f:
        vocab = json.loads(f.read())

    with open("assets/output/contacts.txt", "r", encoding="utf-8") as f:
        contacts = json.loads(f.read())   

    spec_tokens = contacts + [end_token]
    model = torch.load("assets/models/model.pt")
    completer = WordCompleter(spec_tokens, ignore_case=True)
    
    input = prompt("message >> ", completer=completer, default="")
    output = torch.tensor([], dtype=torch.long)
    print()

    while input != end_token:
        for _ in range(n_chats):

            add_tokens = custom_tokenizer(input, spec_tokens)
            add_context = encode(add_tokens, vocab)
            context = torch.cat((output, add_context)).unsqueeze(1).T

            n0 = len(output)
            output = model.generate(context, vocab)
            n1 = len(output)

            print_delayed(decode(output[n0-n1:], vocab))
            input = random.choice(contacts)

        input = prompt("\nresponse >> ", completer=completer, default="")
        print()
        

## model.py

El archivo model.py implementa desde cero una versión simplificada de un modelo GPT (Generative Pretrained Transformer) utilizando PyTorch, siguiendo la arquitectura de los transformadores. Está compuesto por distintos módulos que se combinan jerárquicamente para procesar secuencias de texto y generar predicciones token por token.
Primero, la clase Head define una cabeza de atención individual, que aplica el mecanismo de self-attention: para cada token calcula consultas (queries), claves (keys) y valores (values), genera una matriz de atención con q @ kᵀ / √d, aplica una máscara triangular inferior para evitar mirar tokens futuros (causalidad), y produce una salida ponderada de los valores.
La clase MultiHeadAttention combina varias cabezas de atención en paralelo (según n_heads), concatena sus salidas y las pasa por una capa lineal, replicando el mecanismo de atención múltiple del transformer original.
Luego, la clase FeedForward aplica dos capas lineales con activación ReLU y dropout, expandiendo y luego reduciendo la dimensionalidad del embedding; actúa como una red no lineal por token.
La clase Block representa un bloque completo del transformador, compuesto por atención multi-cabeza seguida de la red feedforward, con residual connections (sumas con la entrada) y LayerNorm antes de cada submódulo, que estabilizan el entrenamiento.
La clase principal GPTLanguageModel construye el modelo completo: incluye embeddings de tokens y de posición (para dar información del orden en la secuencia), una pila secuencial de n_layer bloques transformer, y una capa final lineal para proyectar al tamaño del vocabulario y obtener los logits de probabilidad sobre el siguiente token. Además, inicializa los pesos de manera normal con desviación estándar baja para estabilidad.
En el método forward, el modelo toma una matriz de índices de tokens (idx), obtiene sus embeddings, les suma los embeddings posicionales, los pasa por los bloques y produce los logits. Si se proporcionan targets, calcula la pérdida de entropía cruzada.
Finalmente, el método generate implementa la generación de texto autoregresiva: dado un contexto inicial (idx), predice el siguiente token, lo añade a la secuencia y repite hasta encontrar el token de fin (<END>), evitando seleccionar el token desconocido (<UNK>). Así, el archivo model.py contiene toda la definición estructural del modelo GPT, desde los mecanismos básicos de atención hasta el proceso de generación secuencial de texto.

In [ ]:
import math

import torch
import torch.nn as nn
from torch.nn import functional as F

from config import (block_size, dropout, embed_size, end_token, n_heads,
                    n_layer, unknown_token)
from src.utils import encode


class Head(nn.Module):
    """
    This module performs self-attention operations on the input tensor, producing 
    an output tensor with the same time-steps but different channels. 
    
    :param head_size: The size of the head in the multi-head attention mechanism.
    """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embed_size, head_size, bias=False)
        self.query = nn.Linear(embed_size, head_size, bias=False)
        self.value = nn.Linear(embed_size, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        """
        B,T,C = x.shape
        k = self.key(x)                                     # (B, T, head_size)
        q = self.query(x)                                   # (B, T, head_size)

        # compute attention scores
        wei = q @ k.transpose(-2,-1)                        # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)
        wei /= math.sqrt(k.shape[-1])                       # (B, T, T)
        
        # avoid look-ahead
        tril = torch.tril(torch.ones(T, T))
        wei = wei.masked_fill(tril == 0, float("-inf"))     # (B, T, T)
        wei = F.softmax(wei, dim=-1)                        # (B, T, T)
        wei = self.dropout(wei)
        
        # weighted aggregation of the values
        v = self.value(x)                                   # (B, T, head_size)
        out = wei @ v                                       # (B, T, T) @ (B, T, hs) -> (B, T, head_size)
        return out


class MultiHeadAttention(nn.Module):
    """
    This class contains multiple `Head` objects, which perform self-attention 
    operations in parallel.
    """
    def __init__(self):
        super().__init__()

        # list of parallel heads that are concatenated by the linear layer in the end
        head_size = embed_size // n_heads
        heads_list = [Head(head_size) for _ in range(n_heads)]
        
        self.heads = nn.ModuleList(heads_list)
        self.linear = nn.Linear(n_heads * head_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        heads_list = [h(x) for h in self.heads]
        out = torch.cat(heads_list, dim=-1)
        out = self.linear(out)
        out = self.dropout(out)
        return out


class FeedFoward(nn.Module):
    """
    This module passes the input tensor through a series of linear transformations 
    and non-linear activations.
    """
    def __init__(self):
        super().__init__()
        # factor of 4 is the multiplier of nodes
        self.net = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size), 
            nn.ReLU(),
            nn.Linear(4 * embed_size, embed_size),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """
    This module contains a single transformer block, which consists of multi-head 
    self-attention followed by feed-forward neural networks.
    """
    def __init__(self):
        super().__init__()

        self.sa = MultiHeadAttention()
        self.ffwd = FeedFoward()
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class GPTLanguageModel(nn.Module):
    """
    This class encompasses the entire GPT model, including the token and position embeddings, 
    multiple transformer blocks, and output layer.
    """
    def __init__(self, vocab_size: int):
        super().__init__()

        # embedding tables for token and their positioning in the context
        self.token_embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_embedding = nn.Embedding(block_size, embed_size)
        
        # put one block after the other sequentially (not parallel like multi-head attention)
        block_list = [Block() for _ in range(n_layer)]
        self.blocks = nn.Sequential(*block_list)
        
        # output layer after sequential blocks
        self.ln_output = nn.LayerNorm(embed_size)
        self.linear_output = nn.Linear(embed_size, vocab_size)

        # initialize weights and biases for linear layers and embeddings
        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            
            # The linear layers in self-attention do not have a biases
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding(idx)                     # (B, T, C)
        pos_emb = self.pos_embedding(torch.arange(T))           # (T, C)
        x = tok_emb + pos_emb                                   # (B, T, C)
        x = self.blocks(x)                                      # (B, T, C)
        x = self.ln_output(x)                                   # (B, T, C)
        logits = self.linear_output(x)                          # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, vocab):
        
        # Initialize idx_net for while loop
        idx_next = torch.zeros(1)
        idx_end = encode([end_token], vocab)
        idx_unk = encode([unknown_token], vocab)

        # continue to sample tokens until special end token
        while idx_next[0] != idx_end:

            # idx is (B, T) array of indices in the current context
            # crop idx to the last block_size tokens for each batch (row)
            idx_cond = idx[:, -block_size:]                     # (B, T)

            # get the predictions
            logits, _ = self(idx_cond)                          # (B, T, vocab_size)
            logits = logits[:, -1, :]                           # (B, vocab_size)            
            probs = F.softmax(logits, dim=-1)                   # (B, vocab_size)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # when the sampled token is UNK, then sample again
            while idx_next[0] == idx_unk:
                idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
                
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)             # (B, T+1)

        # output everything except the end token
        return idx[0][:-1]


## preprocess.py

El archivo preprocess.py se encarga de preparar los datos de entrenamiento y validación a partir de un chat de WhatsApp exportado, para que puedan ser usados por el modelo GPT definido en el proyecto. Su flujo principal está en la función make_train_test(), que realiza todo el proceso de limpieza, tokenización y codificación del texto.
Primero, el script abre el archivo assets/input/chat.txt que contiene las conversaciones. Luego, identifica los caracteres poco frecuentes (como emojis o símbolos raros) mediante la función get_infrequent_tokens, que usa un contador (Counter) para marcar aquellos que aparecen menos veces que un umbral (min_count_chars). Esos caracteres infrecuentes se eliminan del texto con drop_chars.
Después, usa una expresión regular (re.findall) para dividir el chat en tuplas con la estructura (fecha, contacto, mensaje) y descarta los mensajes del sistema (que empiezan con \u200e). Los nombres de los contactos se convierten a minúsculas y se tratan como tokens especiales, agregándoles un “:” al final para distinguirlos.
La función flatten_tuple combina esas tuplas en un solo texto plano, donde cada mensaje termina con el token especial de fin (<END>). Luego se tokeniza el texto con custom_tokenizer, que probablemente divide el texto en palabras o subunidades considerando los tokens especiales definidos.
Posteriormente, se detectan los tokens poco frecuentes con get_infrequent_tokens (esta vez a nivel de palabra/token, usando min_count_tokens como umbral) y se reemplazan por el token <UNK> mediante mask_tokens, reduciendo así el tamaño del vocabulario.
Una vez limpio y tokenizado, se genera el vocabulario con get_vocab, que asigna un índice entero a cada token. Con esa codificación, los tokens se transforman a tensores numéricos con encode. Los datos resultantes se dividen en un 90 % para entrenamiento y un 10 % para validación, y se guardan como archivos de PyTorch (train.pt y valid.pt).
Finalmente, se exportan también los archivos vocab.txt (el diccionario de tokens) y contacts.txt (los nombres de los emisores del chat) en formato JSON. Así, preprocess.py automatiza toda la etapa de preparación de datos, dejando listos los tensores y los metadatos que el modelo GPT utilizará para aprender a predecir y generar mensajes similares al estilo del chat original.

In [ ]:
import json
import re
from collections import Counter
from typing import List, Set, Tuple, Union

import torch

from config import end_token, min_count_chars, min_count_tokens, unknown_token
from src.utils import custom_tokenizer, encode, get_vocab


def get_infrequent_tokens(tokens: Union[List[str], str], min_count: int) -> List[str]:
    """
    Identify tokens that appear less than a minimum count.
    
    :param tokens: When it is the raw text in a string, frequencies are counted on character level.
                   When it is the tokenized corpus as list, frequencies are counted on token level.
    :min_count: Threshold of occurence to flag a token.
    :return: List of tokens that appear infrequently. 
    """
    counts = Counter(tokens)
    infreq_tokens = set([k for k,v in counts.items() if v<=min_count])
    return infreq_tokens


def mask_tokens(tokens: List[str], mask: Set[str]) -> List[str]:
    """
    Iterate through all tokens. Any token that is part of the set, is replaced by the unknown token.

    :param tokens: The tokenized corpus.
    :param mask: Set of tokens that shall be masked in the corpus.
    :return: List of tokenized corpus after the masking operation.
    """
    return [t.replace(t, unknown_token) if t in mask else t for t in tokens]


def drop_chars(txt: str, drop: Set[str]) -> str:
    """Drop a list of characters from string"""

    return txt.translate(str.maketrans("", "", "".join(drop)))


def flatten_tuple(txt: List[Tuple[str, str]]) -> str:
    """Convert list of tuples into string separated by the end token"""

    return "".join([x0+":"+x1+end_token for x0, x1 in txt])


def make_train_test() -> None:
    """
    Prepare training and testing datasets from chat messages. This function performs multiple tasks:
    
    1. Reads a corpus of WhatsApp chat messages from a text file
    2. Filters out infrequent characters from the corpus
    3. Splits the text based on regular expressions
    4. Tokenizes the text and encodes the tokens into integers
    5. Splits the encoded data into training and validation sets
    6. Saves the training and validation datasets, as well as the vocab and senders, to disk
    """
    with open("assets/input/chat.txt", "r") as f:
        text = f.read()

    # remove very rare characters (mostly emojies)
    infreq_chars = get_infrequent_tokens(text, min_count=min_count_chars)
    text = drop_chars(text, infreq_chars)

    # split string into list of tuples (date, contact, message)
    pattern = r'\[(.*?)\] (.*?): (.*)'
    matches = re.findall(pattern, text)
    text = [(x1, x2.lower()) for x0, x1, x2 in matches if not x2.startswith("\u200e")]

    # get list of all contacts, treated as special tokens
    contacts = list(set([contact+":" for contact, msg in text]))
    spec_tokens = contacts + [end_token]

    # convert list of tuples into list of tokens (word or character level)
    text_flat = flatten_tuple(text)
    tokens = custom_tokenizer(txt=text_flat, spec_tokens=spec_tokens)

    # mask very rare tokens as unknown, to shrink the vocabulary
    infreq_tokens = get_infrequent_tokens(tokens, min_count=min_count_tokens)
    tokens = mask_tokens(tokens, infreq_tokens)

    # get vocabulary of corpus to file
    vocab = get_vocab(tokens)
    print(f"The corpus has {len(vocab)} unique tokens.")

    # encode tokens into a tensor of integers
    data = encode(tokens, vocab)

    # split up the data into train and validation set
    n = int(0.9*len(data))
    train_data = data[:n]
    valid_data = data[n:]

    # export tensors
    torch.save(train_data, "assets/output/train.pt")
    torch.save(valid_data, "assets/output/valid.pt")

    with open("assets/output/vocab.txt", "w", encoding="utf-8") as f:
        f.write(json.dumps(vocab))

    with open("assets/output/contacts.txt", "w", encoding="utf-8") as f:
        f.write(json.dumps(contacts))

    print("SUCCESS")


## train.py

El archivo train.py contiene el módulo encargado de entrenar o actualizar el modelo GPT usando los datos procesados previamente. Su función principal, model_training(update: bool), controla todo el flujo de entrenamiento.
Primero, el script carga los tensores de entrenamiento y validación (train.pt y valid.pt) junto con el vocabulario (vocab.txt) que contiene el mapeo entre tokens y sus índices numéricos. Luego, según el valor del parámetro update, decide si inicia un nuevo modelo (GPTLanguageModel) o carga uno previamente entrenado desde assets/models/model.pt para continuar el aprendizaje. Esto permite tanto un entrenamiento desde cero como una reanudación incremental.
Después de definir el modelo, se crea un optimizador AdamW con una tasa de aprendizaje (learn_rate) definida en el archivo config.py. El script también calcula y muestra el número total de parámetros entrenables del modelo para tener una idea del tamaño de la red.
En el bucle principal de entrenamiento, que se ejecuta durante max_iters iteraciones, cada cierto número de pasos (determinado por eval_interval), se evalúa el desempeño del modelo midiendo la pérdida (loss) en los conjuntos de entrenamiento y validación usando la función auxiliar estimate_loss. Esta función realiza una evaluación rápida sin actualizar los pesos, permitiendo monitorear si el modelo mejora o se sobreajusta.
En cada iteración, se extrae un lote de ejemplos mediante get_batch(train_data), que selecciona fragmentos de texto (secuencias de tokens) y sus correspondientes objetivos de predicción. Luego, el modelo calcula los logits y la pérdida (loss) comparando sus predicciones con los objetivos reales.
La pérdida se retropropaga con loss.backward(), el optimizador actualiza los pesos (optimizer.step()), y antes de cada paso se limpian los gradientes acumulados con optimizer.zero_grad(). Este ciclo de cálculo, retropropagación y actualización se repite hasta completar todas las iteraciones.
Finalmente, el modelo entrenado se guarda en disco (model.pt) para poder usarse después en la generación de texto (por ejemplo, en el modo “chat”).
En resumen, train.py implementa todo el proceso de entrenamiento supervisado del modelo GPT, gestionando desde la carga de datos y configuración hasta la optimización iterativa y el guardado del modelo final.

In [ ]:
import json

import torch

from config import eval_interval, learn_rate, max_iters
from src.model import GPTLanguageModel
from src.utils import current_time, estimate_loss, get_batch


def model_training(update: bool) -> None:
    """
    Trains or updates a GPTLanguageModel using pre-loaded data.

    This function either initializes a new model or loads an existing model based
    on the `update` parameter. It then trains the model using the AdamW optimizer
    on the training and validation data sets. Finally the trained model is saved.

    :param update: Boolean flag to indicate whether to update an existing model.
    """
    # LOAD DATA -----------------------------------------------------------------

    train_data = torch.load("assets/output/train.pt")
    valid_data = torch.load("assets/output/valid.pt")

    with open("assets/output/vocab.txt", "r", encoding="utf-8") as f:
        vocab = json.loads(f.read())

    # INITIALIZE / LOAD MODEL ---------------------------------------------------

    if update:
        try:
            model = torch.load("assets/models/model.pt")
            print("Loaded existing model to continue training.")
        except FileNotFoundError:
            print("No existing model found. Initializing a new model.")
            model = GPTLanguageModel(vocab_size=len(vocab))
        
    else:
        print("Initializing a new model.")
        model = GPTLanguageModel(vocab_size=len(vocab))

    # initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learn_rate)

    # number of model parameters
    n_params = sum(p.numel() for p in model.parameters())
    print(f"Parameters to be optimized: {n_params}\n", )

    # MODEL TRAINING ------------------------------------------------------------

    for i in range(max_iters):

        # evaluate the loss on train and valid sets every 'eval_interval' steps
        if i % eval_interval == 0 or i == max_iters - 1:
            train_loss = estimate_loss(model, train_data)
            valid_loss = estimate_loss(model, valid_data)

            time = current_time()
            print(f"{time} | step {i}: train loss {train_loss:.4f}, valid loss {valid_loss:.4f}")

        # sample batch of data
        x_batch, y_batch = get_batch(train_data)

        # evaluate the loss
        logits, loss = model(x_batch, y_batch)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    torch.save(model, "assets/models/model.pt")
    print("Model saved")


## utils.py

El archivo utils.py reúne un conjunto de funciones auxiliares fundamentales para el funcionamiento general del sistema GPT, cubriendo desde el manejo de datos y tokenización hasta la evaluación del modelo y la impresión de resultados.
La función estimate_loss(model, data) evalúa el rendimiento del modelo sin modificar sus pesos. Para ello, cambia el modo del modelo a evaluación (model.eval()), calcula la pérdida en varios lotes (eval_iters veces) usando la función get_batch, promedia los resultados y luego devuelve el modelo a modo de entrenamiento (model.train()). Esto permite medir la calidad del aprendizaje sin interferir con el entrenamiento.
La función get_batch(data) crea pequeños lotes de entrada y salida a partir de los datos codificados. Genera índices aleatorios dentro del tensor completo y toma fragmentos de longitud block_size. El tensor x contiene los tokens actuales, y y es el mismo desplazado una posición hacia adelante, es decir, los targets que el modelo debe predecir.
La función encode(s, vocab) transforma una lista de tokens en un tensor de índices enteros, usando el vocabulario. Si un token no se encuentra, se reemplaza por el token especial <UNK> (definido en config.py). Si ese token no existe en el vocabulario (por ejemplo, en datos nuevos), se le asigna un índice aleatorio, garantizando que todos los elementos tengan representación numérica.
La inversa, decode(tensor, vocab), convierte un tensor de índices nuevamente en texto legible, usando un mapeo inverso de índices a tokens y uniéndolos con espacios.
El custom_tokenizer es un tokenizador configurable basado en RegexpTokenizer de NLTK. Su patrón por defecto divide el texto en palabras o caracteres, pero además protege los tokens especiales (spec_tokens) —como nombres de contactos o el token de fin <END>— para que se mantengan intactos durante la segmentación.
La función get_vocab(text) genera una lista ordenada de los tokens únicos del corpus, que se usa para construir el diccionario de vocabulario del modelo.
current_time() devuelve la hora actual en formato HH:MM:SS, usada principalmente para registrar avances durante el entrenamiento.
Finalmente, print_delayed(s, delay) imprime una cadena carácter por carácter con una pequeña pausa configurable, simulando una respuesta “en tiempo real” durante el modo de chat interactivo.
En conjunto, este módulo proporciona las herramientas de soporte esenciales para el preprocesamiento, la codificación de texto, la evaluación del modelo y la interfaz interactiva, sirviendo de base para la comunicación entre los distintos componentes del sistema GPT.

In [ ]:
import random
import time
from datetime import datetime
from typing import List, Union

import torch
from nltk.tokenize import RegexpTokenizer

from config import batch_size, block_size, eval_iters, unknown_token


@torch.no_grad()
def estimate_loss(model, data):
    """
    Set evaluation mode and evaluate the loss on multiple batches. 
    Return the average of collected losses.
    """
    model.eval() 
    loss_list = torch.zeros(eval_iters)
    
    for i in range(eval_iters):
        X, Y = get_batch(data)
        logits, loss = model(X, Y)
        loss_list[i] = loss.item()

    loss_avg = loss_list.mean()    
    model.train() 
    return loss_avg


def get_batch(data):
    """Generate a small batch of data of inputs x and targets y"""

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


def encode(s: list, vocab: list) -> torch.tensor:
    """
    Encode a list of tokens into a tensor of integers, given a fixed vocabulary. 
    When a token is not found in the vocabulary, the special unknown token is assigned. 
    When the training set did not use that special token, a random token is assigned.
    """
    rand_token = random.randint(0, len(vocab))

    map = {s:i for i,s in enumerate(vocab)}
    enc = [map.get(c, map.get(unknown_token, rand_token)) for c in s]
    enc = torch.tensor(enc, dtype=torch.long)
    return enc


def decode(tensor: torch.tensor, vocab: list) -> str:
    """Decode a tensor of integers, back into a string."""

    map_enc = {s:i for i,s in enumerate(vocab)}
    map_dec = {i:s for s,i in map_enc.items()}
    dec = [map_dec[i.item()] for i in tensor]
    dec = " ".join(dec)
    return dec


def custom_tokenizer(txt: str, spec_tokens: List[str], pattern: str="|\d|\\w+|[^\\s]") -> List[str]:
    """
    Tokenize text into words or characters using NLTK's RegexpTokenizer, considerung 
    given special combinations as single tokens.

    :param txt: The corpus as a single string element.
    :param spec_tokens: A list of special tokens (e.g. ending, out-of-vocab).
    :param pattern: By default the corpus is tokenized on a word level (split by spaces).
                    Numbers are considered single tokens.

    >> note: The pattern for character level tokenization is '|.'
    """
    pattern = "|".join(spec_tokens) + pattern
    tokenizer = RegexpTokenizer(pattern)
    tokens = tokenizer.tokenize(txt)
    return tokens


def get_vocab(text: Union[List[str], str]) -> List[str]:
    """Returns a sorted list of all unique tokens in the corpus."""

    return sorted(list(set(text)))


def current_time():
    return datetime.now().strftime("%H:%M:%S")


def print_delayed(s: str, delay: float = 0.05) -> None:
    """
    Prints each character of a string one by one on the same line with a delay.

    :param s: The input string.
    :param delay: The time delay between each character in seconds.
    """
    for char in s:
        print(char, end="", flush=True)
        time.sleep(delay)

    print()
